In [2]:
import matplotlib.pyplot as plt

from matplotlib.patches import Rectangle

import h5py as h5

import numpy as np

import gvar as gv

from tqdm.auto import tqdm

from lsqfit import nonlinear_fit

from copy import deepcopy

from itertools import product

from pathlib import Path

import lib_perylene as perylene

import plotting

import fitter

import scipy.stats as sps

%matplotlib inline

ModuleNotFoundError: No module named 'fitModels'

In [ ]:
class ParamSpace():
    def __init__(self, Nts = None, betas = None, Us = None, mus = None):
        if Nts is None:
            self.Nts = np.array([32,64,96])
            # self.Nts = np.array([64,96])
        else:
            self.Nts = Nts
            
        if betas is None:
            self.betas = np.array([4,6,8])
            # self.betas = np.array([6,8])
        else:
            self.betas = betas
            
        if Us is None:
            self.Us = [2]
        else:
            self.Us = Us
        
        if mus is None:
            self.mus = np.arange(21)*0.1
        else:
            self.mus = mus
        
        self.Nparams = len(self.Nts)*len(self.betas)*len(self.Us)*len(self.mus)

    def items(self):
        return product(
            self.Nts, self.betas, self.Us, self.mus
        )

    def toKey(self,Nt,beta,U,mu):
        return f"Nt{Nt:g}/beta{beta:g}/U{U:g}/mu{mu:g}"
   
Nx = 20
Nfit = 40
Nbst = 100
U = 2

reportFolder = Path("./ReportNbst")
CacheFolder  = Path("./CacheNbst")

paramSpace = ParamSpace(mus = np.arange(11)*0.1)
paramRelevant = paramSpace #ParamSpace(mus = np.arange(6)*0.1, Nts = [32,64])

In [ ]:
def QNonInt(mu,beta = None):
    ev = perylene.quantum_numbers[:,2] + mu
    
    if beta is None:
        return 2 * np.sum(
            np.heaviside( ev, 0.5 )
        ) - Nx
    else:
        return 2 * np.sum(
            1/(np.exp( -beta * ev ) + 1)
        ) - Nx
    

In [ ]:
def GrubbsTestStatistic(data,mean = None, std = None, alpha = 0.01) -> (int,float,bool):
    r"""
        This is adapted from 
        https://en.wikipedia.org/wiki/Grubbs%27s_test#Definition
        G = (max_{i} | X_i - \mu |) / (\sigma)
        with mean = \mu and std = \sigma
        we then identify if i for which the maximum was found is a outlier by 
        G > (N-1)/sqrt(N) * sqrt( t^2 / (N-2 + t^2)  )
        where t denotes the critical value of the t-distribution with N-2 dof and a signficance level of $\alpha/2N$
        return OutlierIndex[int], GStatistic[float], isOutlier[bool] 
    """
    if data.ndim != 1: raise RuntimeError("Grubbs test only implemented for 1D data")
    N = data.size 
    
    signficanceLevel = alpha / (2.0*N) 
    t = sps.t.isf(signficanceLevel, N - 2)
    G_target = ((N-1)/np.sqrt(N)) * np.sqrt( t**2 / (N-2+t**2) )

    if mean is None:
        mean = data.mean()
    if std is None:
        std  = data.std()
    
    diff  = np.abs(data - mean)
    
    imax = np.argmax( diff )

    G = diff[imax] / std

    imaxIsOutlier = G > G_target

    return imax, G, imaxIsOutlier

In [ ]:
energies = {
        f"beta{beta:g}/mu{mu:g}": {
        "est": np.zeros((Nx,len(paramRelevant.Nts))),
        "err": np.zeros((Nx,len(paramRelevant.Nts))),
    }

    for beta,mu in product(paramRelevant.betas,paramRelevant.mus)
}
    
for beta,mu,(NtID,Nt) in (pbar:=tqdm(product(paramRelevant.betas,paramRelevant.mus,enumerate(paramRelevant.Nts)), total = len(list(paramSpace.items())))):
    ensemblePath = Path(f"Nt{Nt:g}/beta{beta:g}/U{U:g}/mu{mu:g}")

    fits_ = { f'irrepID{irrepID}':None for irrepID in range(Nx) }

    for irrepPath in (CacheFolder/ensemblePath).iterdir():
        irrepID = irrepPath.name
        
        files = list(enumerate(irrepPath.iterdir()))
        numFits = len(files)
        
        if numFits == 0:
            print(f"No fits done for Nt{Nt:g}/beta{beta:g}/U2/mu{mu:g}/{irrepID}")
            continue
            
        irrepID = int(irrepID[7:])
            
        E_bst = np.zeros( (Nbst,numFits) )        
        E_est = np.zeros( numFits )
        E_err = np.zeros( numFits )
        
        AIC_bst = np.zeros( (Nbst,numFits) )        
        AIC_est = np.zeros( numFits )
        
        noBst = False
        for iFile,file in files:
            with h5.File(file) as h5f:
                _, Nstate, ts, te = str(file.name).split('_')
                Nstate = int(Nstate[6:])                
                try:
                    E_bst[:,iFile] = h5f['FitResult/bst/E0'][()] if perylene.decayingCorr[f'mu{mu:g}/beta{beta:g}'][f'irrepID{irrepID}'] else -h5f['FitResult/bst/F0'][()]
                    AIC_bst[:,iFile] =  h5f['FitStatistic/AIC/bst'][()]
                except Exception as e:
                    #pbar.write(f"Failed to load bootstrap: {file}")
                    noBst = True
                    #raise e
                    
                E_est[iFile]   = h5f['FitResult/est/E0'][()] if perylene.decayingCorr[f'mu{mu:g}/beta{beta:g}'][f'irrepID{irrepID}'] else -h5f['FitResult/est/F0'][()]
                
                E_err[iFile] = h5f['FitResult/err/E0'][()] if perylene.decayingCorr[f'mu{mu:g}/beta{beta:g}'][f'irrepID{irrepID}'] else h5f['FitResult/err/F0'][()]

                AIC_est[iFile] =  h5f['FitStatistic/AIC/est'][()]
        # end for iFile,file
        # Remove outliers
        while True:
            imax, G, isOutlier = GrubbsTestStatistic(E_est, alpha = 0.05)

            if isOutlier:
                #print(f"Found Outlier ({irrepID=}); Deleted index={imax}, {G=:g}: E={E_est[imax]:g}")
                E_est = np.delete( E_est, imax )
                E_err = np.delete( E_err, imax )
                AIC_est  = np.delete( AIC_est , imax )

                if not noBst:
                    E_bst = np.delete( E_bst, imax, axis = 1 )
                    AIC_bst  = np.delete( AIC_bst, imax, axis = 1 )
            else:
                break

        if noBst:
            modelAvg = fitter.modelAverage_raw(
                param_est = E_est, AIC_est = AIC_est,
                param_err = E_err,
            )             
        else:
            modelAvg = fitter.modelAverage_raw(
                param_est = E_est, AIC_est = AIC_est,
                param_bst = E_bst, AIC_bst = AIC_bst
            ) 
        energies[f"beta{beta:g}/mu{mu:g}"]['est'][irrepID,NtID] = modelAvg['est']
        energies[f"beta{beta:g}/mu{mu:g}"]['err'][irrepID,NtID] = modelAvg['err']
# end for beta,mu,(NtID,Nt)


In [ ]:
from scipy.optimize import curve_fit

def continuumLimit(energies,paramSpace,U = 2):
    Nx = 20
    
    def model(delta,p):
        r"""
            delta: np.array of lattice spacings
            p    : parameters:
                p = {
                    E0: y-intercept, continuum value
                    E1: linear with delta
                    E2: quadratic with delta 
                    ...
                }
        """

        out = np.ones_like(delta,dtype=object)
        for key in p.keys():
            power = int(key[1])
            if power == 0: continue
            out += delta**power * p[key]
        out *= p['E0']
        return out

    def getP0(beta,mu,irrepID,terms):
        r"""
            Get the priors for the above fitmodel:
            
            mu: Chemical potential value
            irrep: integer 0,1...,19 identifying the irreducible repr.
            terms: List of terms ['E0','E1','E2',...]
        """

        p0 = gv.BufferDict()
        
        
        for key in terms:
            if key == 'E0':
                p0[key] = energies[f'beta{beta:g}/mu{mu:g}']['est'][irrepID][-1]
                # perylene.quantum_numbers[irrepID,2] + mu
            else:
                p0[key] = 0

        return p0
        
    results = {
        f"beta{beta:g}":{
            "Limits": np.empty( (len(paramSpace.mus), Nx), dtype=object ),
            "Econt": np.empty(  (len(paramSpace.mus), Nx), dtype=object ),
        }
        for beta in paramSpace.betas
    }

    for (i,mu),beta in product(enumerate(paramSpace.mus), paramSpace.betas):
        

        for irrepID in range(Nx):
            deltas = beta / np.array(paramSpace.Nts)
            ordinate = gv.gvar(energies[f'beta{beta:g}/mu{mu:g}']['est'][irrepID], energies[f'beta{beta:g}/mu{mu:g}']['err'][irrepID])
            
            # E(δ) = E₀ + δ E₁
            fit_1 = nonlinear_fit(
                udata  = (deltas, ordinate), 
                fcn   = model, 
                p0 = getP0(beta,mu,irrepID,['E0']) 
            )

            results[f"beta{beta:g}"]["Limits"][i,irrepID] = fit_1
            results[f"beta{beta:g}"]["Econt"][i,irrepID] = fit_1.p['E0'] #0.5 * ( fit_1.p['E0'] + fit_2.p['E0'])
            # results[f"beta{beta:g}"]["Econt"][i,irrepID] = (1/3) * ( np.sum(ordinate) )
    return results

results = continuumLimit(energies,paramRelevant)


In [ ]:
def plotEnergyCont(results, paramSpace):
    Nx = 20
    Nplot = 20

    numRows = len(paramSpace.mus)
    numCols = len(paramSpace.betas)
    colors = list(plotting.style.COLORS.values())
    colorIndex = lambda k: (k * 7) % len(colors)

    irreps = range(Nx)

    pbar = tqdm( total = len(paramSpace.betas)*len(paramSpace.mus)*Nx )
    for irrepID in irreps:
        fig,axs = plt.subplots(
            numRows,
            numCols,
            figsize=(18.5, 26.12), # DIN A2
            #figsize = (23.3, 33.1), # DIN A1
            sharex = 'col',
            sharey = 'row',
            squeeze = False
        )

        handles = [None] * len(paramSpace.betas)

        colorKey = ['primary', 'complementary', 'highlight']
        #NstateMarker = [ 'o','x','^'] # [r'$1$',r'$2$',r'$3$']

        for ibeta, beta in enumerate(paramSpace.betas):
            for imu,mu in enumerate(paramSpace.mus):
                fit = results[f"beta{beta:g}"]["Limits"][imu,irrepID]
                if fit is None:
                    print(f"beta={beta:g}, mu={mu:g}")
                    print(results[f"beta{beta:g}"]["Econt"])
                    continue
                Econt = results[f"beta{beta:g}"]["Econt"][imu,irrepID]
                deltas = fit.x
                Energy = fit.y
                dof = fit.dof
                chi2dof= fit.chi2/dof

                abscissa = np.linspace(0,np.max(deltas),Nplot)
                ordinate = fit.fcn(abscissa,fit.p)

                # Plot best fit:
                axs[imu,ibeta].plot(
                    abscissa, gv.mean(ordinate), '-', color = plotting.style.MAIN_COLORS[colorKey[ibeta]],
                    label = rf"$^{{\chi^2}}/_\mathrm{{dof}} = {chi2dof:g}$"
                )
                axs[imu,ibeta].fill_between(
                    abscissa,
                    gv.mean(ordinate)+gv.sdev(ordinate),
                    gv.mean(ordinate)-gv.sdev(ordinate),
                    color = plotting.style.MAIN_COLORS[colorKey[ibeta]],
                    alpha = 0.2
                )

                # Plot Continuum Value
                axs[imu,ibeta].errorbar(
                    [0], [Econt.mean], yerr=[Econt.sdev],
                    capsize = 2,
                    fmt = '^',
                    color = colors[colorIndex(irrepID)],
                )

                # Plot the initial data
                for delta,E in zip( deltas,Energy ):
                    Nt = int( beta/delta  )
                    marker = 'o'

                    axs[imu,ibeta].errorbar(
                        [delta], [gv.mean(E)], yerr=[gv.sdev(E)],
                        capsize = 2,
                        marker = marker,
                        color = plotting.style.MAIN_COLORS[colorKey[ibeta]],
                    )

                # Set labels

                # axs[imu,ibeta].set_title(
                #     rf"$\beta = {beta:g}, \, \mu = {mu:g}, \, ^{{\chi^2}}/_\mathrm{{dof}} \, [\mathrm{{dof}}] = {chi2dof:g} \, [{dof}]$",
                #     fontsize=18
                # )
                axs[imu,ibeta].legend(fontsize = 18, loc = 'upper right')
                axs[imu,0].set_ylabel(r'$E(\delta)$',fontsize = 20)
                axs[imu,2].set_ylabel(rf'$\mu = {mu:g}$',fontsize = 20)
                axs[imu,2].yaxis.set_label_position("right")
                axs[imu,ibeta].tick_params(axis='both', which='both', labelsize=18)
                pbar.update(1)
            # end for imu,mu
            axs[-1,ibeta].set_xlabel(r'$\delta$'   ,fontsize = 20)
        # end for ibeta, beta
        axs[0,0].set_title(rf"$\beta = {4:g}$",fontsize=20)
        axs[0,1].set_title(rf"$\beta = {6:g}$",fontsize=20)
        axs[0,2].set_title(rf"$\beta = {8:g}$",fontsize=20)
        fig.tight_layout()
        plotPath = Path(reportFolder)/'ContinuumLimits'
        plotPath.mkdir(parents=True,exist_ok=True)
        fig.savefig(plotPath/f"energyContLimit_irrepID{irrepID}.pdf", dpi=10)
        plt.close(fig)
    # end for irrepID
# end plotEnergyCont

plotEnergyCont(results, paramRelevant)

In [ ]:
def plotEnergyContPerMu(results, paramSpace, U = 2):
    Nx = 20 
    Nplot = 20
    
    numCols = 6 
    numRows = 4
    colors = list(plotting.style.COLORS.values())
    colorIndex = lambda k: (k * 7) % len(colors)


    irreps = range(Nx)

    pbar = tqdm( total = len(paramSpace.betas)*len(irreps) )
    colorKey = ['primary', 'complementary', 'highlight']

    for ibeta, beta in enumerate(paramSpace.betas):
        mus = paramSpace.mus
            
        fig,axs = plt.subplots(
            numRows,
            numCols,
            figsize=(23.3, 16.5), # DIN A2
            squeeze = False
        )

        for irrepID in irreps:
            if irrepID in [0,1,2,3,4,5]: 
                row,col = 0,irrepID
            elif irrepID in [6,7,8,9]: 
                row,col = 1, irrepID-6
                axs[row,4].set_axis_off()
                axs[row,5].set_axis_off()
                # axs[1,6].set_axis_off()
            elif irrepID in [10,11,12,13,14,15]:
                row,col = 2, irrepID-10                
            elif irrepID in [16,17,18,19]: 
                row,col = 3, irrepID-16
                axs[row,4].set_axis_off()
                axs[row,5].set_axis_off()
                # axs[1,5].set_axis_off()
                
            Econt = results[f"beta{beta}"]["Econt"][:,irrepID]
            
            axs[row,col].errorbar(
                mus, gv.mean(Econt), yerr = gv.sdev(Econt), fmt = '^:', color = colors[colorIndex(irrepID)],capsize=2,
                label = r"$U=2$"
            )

            abscissa = np.linspace(np.min(mus), np.max(mus), Nplot)
            axs[row,col].plot( 
                abscissa, perylene.quantum_numbers[irrepID,2] + abscissa, 'k-',
                label = r"$U=0$"
            )

            axs[row,col].set_xlabel(r'$\mu$', fontsize = 16)
            axs[row,col].set_ylabel(r'$E_\mathrm{cont}(\mu)$', fontsize = 16)
            axs[row,col].set_title( rf"{perylene.D2irreps[irrepID]}",fontsize=18 )
            axs[row,col].legend(fontsize=14)

            pbar.update(1)
        
        fig.tight_layout()
        plotPath = Path(reportFolder)/'ContinuumLimits'
        plotPath.mkdir(parents=True,exist_ok=True)
        fig.savefig(plotPath/f"EcontPerMu_beta{beta:g}.pdf", dpi=10)
        plt.close(fig)
    # end for ibeta,beta 
# end plotEnergyCont


print("Plotting Continuum limit of E...")
plotEnergyContPerMu(results, paramRelevant)
print("Done")

In [ ]:
def plotSpectrum(results,paramSpace):

    colors = list(plotting.style.COLORS.values())
    colorIndex = lambda k: (k * 7) % len(colors)

    for ibeta, beta in enumerate(paramSpace.betas):

        fig,axs = plt.subplots(
            len(paramSpace.mus)+1,
            1,
            #figsize=(12,24),
            figsize=(12, 24),
            sharex='col'
        )
        
        handles = []
    
        sort = np.argsort(perylene.quantum_numbers[:,2]) 
        it = reversed(colors)
        for ik,(k,key) in enumerate(zip(sort,it)):
            axs[0].plot(
                perylene.quantum_numbers[k,2], ik, '^', color =  colors[colorIndex(k)]
            )
        axs[0].set_title("Non Interacting", fontsize = 22)
        axs[0].set_ylabel(fr"$\mu={0:g}$", fontsize = 20)
        axs[0].set_yticks([])
        axs[0].xaxis.set_tick_params(which='both', labelbottom=True)
        axs[0].grid()
        axs[1].set_title(rf"Interacting, $\beta={beta}$", fontsize = 22)
        
        for imu, mu in enumerate( paramSpace.mus ):        
            row = imu + 1
    
            axs[row].axvline(
                0, linestyle='-.', color = 'k' 
            )
        
            for ik,(k,key) in enumerate(zip(sort,colors)):
                fit   =  results[f'beta{beta}']['Limits'][imu,k]
                E_est =  results[f'beta{beta}']['Econt'][imu,k].mean
                E_err =  results[f'beta{beta}']['Econt'][imu,k].sdev
                
                handle, = axs[row].plot(
                    E_est,ik, '',
                    color = colors[colorIndex(k)],
                    label = perylene.D2irreps[k],
                )
                
                axs[row].add_artist(Rectangle(
                    xy = (E_est - E_err, ik-1.5),
                    width = 2*E_err,
                    height = 3,
                    color = colors[colorIndex(k)],
                    alpha = 0.5
                ))
    
                if mu == 0:
                    handles.append(handle)
            axs[row].set_ylabel(fr"$\mu = {mu:g}$", fontsize = 20)
            axs[row].set_yticks([])
            axs[row].set_ylim(-2,21)
            axs[row].xaxis.set_tick_params(which='both', labelbottom=True)
            axs[row].grid()
    
        handles_v2 = deepcopy(handles);
        [h.set_marker('s') for h in handles_v2]
        
        axs[-1].set_xlabel(r"$E^{\Lambda_i}$", fontsize = 20)
        lgd = fig.legend(handles = handles,loc='upper center', 
            bbox_to_anchor=(0.5, -0.01),fancybox=True, shadow=True, ncol=8, fontsize = 20)
    
        fig.tight_layout()
        plotPath = Path(reportFolder)/'ContinuumLimits'
        plotPath.mkdir(parents=True,exist_ok=True)
        fig.savefig(plotPath/f"spectrum_beta{beta:g}.pdf", dpi=10, bbox_inches='tight')
        plt.clf()
        # plt.close()
    # end for ibeta,beta
# end plotSpectrum

print("Plotting...")
plotSpectrum(results, paramRelevant );
print("Done.")

In [ ]:
import pandas as pd

def formatData(results,beta, subset = np.s_[:]):

    df = pd.DataFrame( results[f"beta{beta}"]["Econt"][:,subset], index= [f"{mu:g}" for mu in paramRelevant.mus], columns = perylene.D2irreps[subset])
    
    df.index.name=r'$\mu$'
    df = df.style.set_caption(rf"Summary of $E_\mathrm{{cont}}$ at $\beta={beta:g}$").set_table_styles([{
        'selector': 'caption',
        'props': [
            ('font-size', '20px')
        ]
    }])

    return df

In [ ]:
formatData(results,beta=4)

In [ ]:
print(
    formatData(results,beta=4, subset = np.s_[0 :10]).to_latex(),
    formatData(results,beta=4, subset = np.s_[10:20]).to_latex()
)

In [ ]:
formatData(results,beta=6)

In [ ]:
print(
    formatData(results, beta=6, subset=np.s_[0:10]).to_latex(),
    formatData(results, beta=6, subset=np.s_[10:20]).to_latex(),
)

In [ ]:
formatData(results,beta=8)

In [ ]:
print(
    formatData(results,beta=8, subset = np.s_[0 :10]).to_latex(),
    formatData(results,beta=8, subset = np.s_[10:20]).to_latex()
)
